In [1]:
from pytube import YouTube
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import numpy as np
import wave
import pandas as pd
import os
import pickle

In [7]:
driver = webdriver.Chrome
base = 'https://www.youtube.com/results?search_query='
qstring = '+motorcycle+exhaust+sound'
video_dict={}
make_list=['harley+davidson',
           'kawasaki']

driver = webdriver.Chrome()

for make in make_list:
    url = base+make+qstring
    driver.get(url);
    page = driver.get(url)
    for i in range(3):
    #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
    
        #Wait for page to load
        time.sleep(3)
    soup=bs(driver.page_source,'html.parser')
    videolist=[]
    for video in soup.find_all('a', id='video-title'):
        tmp = 'https://youtube.com' + video.get("href")
        videolist.append(tmp)
    video_dict[make]=videolist.copy()

In [ ]:
for make in make_list:
    for i,url in enumerate(video_dict[make]):
        print(make,i)
        print(url)
        print(type(url))
        try:
            video = YouTube(url)
            video.streams.filter(only_audio=True,mime_type='audio/mp4').order_by('abr').desc().first().download(
            output_path = 'audio'+'/'+make, filename=str(i))
        except:
            print('^This one is not working')
            continue

In [ ]:
def chunk_audio_fft(audio_array,sample_rate,chunk_duration):
    chunked_df = pd.DataFrame()
    num_samples = int(chunk_duration*sample_rate)
    num_chunks = int(len(audio_array)//num_samples)
    for i in range (0,num_chunks,1):
        start_point=i*num_samples
        end_point=(i+1)*num_samples
        chunked_df[i] = fft_normalizer(fft_for_audio(audio_array[start_point:end_point],sample_rate))
    return chunked_df.transpose()

In [ ]:
def fft_for_audio(array_in,sample_rate):
    fft_array = np.fft.fft(array_in)/len(array_in)
    fft_array = fft_array[range(int(len(array_in)/2))]
    frequencies = [i*sample_rate/len(array_in) for i in range(len(fft_array))][1:]
    return pd.Series(abs(fft_array[1:]), index = frequencies)

In [ ]:
def fft_normalizer(fft_in):
    fft_out = fft_in.copy()
    fft_out.index=np.log2(fft_out.index)
    peaks=fft_out.copy()
    fundamental_freq = 0
    fundamental_peak = 0
    for freq in peaks.index:
        if peaks[freq] > ((freq-fundamental_freq)*2+1)*fundamental_peak:
            fundamental_peak = peaks[freq]
            fundamental_freq = freq
    fft_out.index=fft_out.index-fundamental_freq
    new_index=np.linspace(-5,4,num=451)
    resampling_dict = {}
    
    for freq in new_index:
        resampling_dict[freq] = [0,0]
    for freq in fft_out[fft_out.index < new_index.max()].index:
        new_freq = min(new_index, key=lambda x:abs(x-freq))
        resampling_dict[new_freq][0]+=fft_out[freq]
        resampling_dict[new_freq][1]+=1
    for freq in new_index:
        if resampling_dict[freq][1] == 0:
            resampling_dict[freq][1] = 1
    normalized=pd.Series([resampling_dict[freq][0]/resampling_dict[freq][1] for freq in new_index], index=new_index)
    scale = 1/normalized.max()
    return normalized*scale

In [ ]:
total_df=pd.DataFrame()
directory_list = [dirname for dirpath, dirname, files in os.walk('./audio')][0]
for directory_name in directory_list:
    make_df=pd.DataFrame()
    file_list = [files for dirpath, dirname, files in os.walk('./audio/'+directory_name)][0]
    for file_name in file_list:
        try:
            command = 'rm ./tmp.wav'
            os.system(command)
            command = 'ffmpeg -i ./audio/'+directory_name+'/'+file_name+' -ac 1 ./tmp.wav'
            print(command)
            os.system(command)
            # Read file to get buffer                                                                                               
            ifile = wave.open('tmp.wav')
            samples = ifile.getnframes()
            sample_rate = ifile.getframerate()
            audio = ifile.readframes(samples)

            # Convert buffer to float32 using NumPy
            audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
            audio_as_np_float32 = audio_as_np_int16.astype(np.float32)

            # Normalize float32 array so that values are between -1.0 and +1.0                                                      
            max_int16 = 2**15
            audio_normalized = audio_as_np_float32 / max_int16
        
            #Chunk the data into 1 second samples and get normalized FFTs thereof
            chunked_df = chunk_audio_fft(audio_normalized,sample_rate,1)
            make_df = pd.concat([make_df,chunked_df],ignore_index=True)
        except:
            print('^This one is not working.')
            continue
    make_df['make'] = directory_name
    total_df = pd.concat([total_df,make_df],ignore_index=True)
    
with open('train_motorcycles.pickle', 'wb') as to_write:
        pickle.dump(total_df, to_write)
to_write.close()